# 알츠하이머 예측 및 치료 방안 모색
## 목표
### 1. 알츠하이머의 유/무를 예측하는 모델 제작
### 2. 알츠하이머의 유/무에 영향을 미치는 요인을 분석하여 예방 및 치료 방안 모색

## 데이터 출처
- 알츠하이머 데이터
- https://www.kaggle.com/datasets/rabieelkharoua/alzheimers-disease-dataset

# 데이터 확인
## 1. 인적 정보
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|PatientID|식별 번호|471-6900|Age|나이|60-90|
|Gender|성별|0:남자, 1 : 여자|Ethnicity|인종|0:백인, 1 : 아프리카계 미국인, 2 : 아시아, 3 : 기타|
|EducationLevel|교육 수준|0 : 없음, 1 : 고등학교, 2 : 학사, 3 : 더 높음|

## 2. lifestype
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|Smoking|흡연 유/무|0 : 아니오, 1 : 예|AlcoholConsumption|주당 알코올 소비량|0-20|
|PhysicalActivity|주간 신체 활동량|0-10|DietQuality|식단 품질|0-10|
|SleepQuality|수면의 질|4-10|

## 3. 병력
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|FamilyHistoryAlzheimers|알츠하이머 가족력|0 : 없음, 1 : 예|CardiovascularDisease|심혈관 질환 유무|0 : 없음, 1 : 예|
|Diabetes|당뇨병 유무|0 : 없음, 1 : 예|Depression|우울증 유무|0 : 없음, 1 : 예|
|HeadInjury|두부 손상 유무|0 : 없음, 1 : 예|Hypertension|고혈압 유무|0 : 없음, 1 : 예|

## 4. 임상 측정
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|SystolicBP|수축기 혈압|90-180|DiastolicBP|이완기 혈압|60-120|
|CholesterolTotal|콜레스테롤 총량|150-300|CholesterolLDL|LDL콜레스테롤 수치|50-200|
|CholesterolHDL|HDL콜레스테롤 수치|20-100|CholesterolTriglycerides|중성지방|50-400|
|BMI|신체질량지수|15-40|

## 5. 인지 및 기능 평가
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|MMSE|간이 정신 상태 검사 점수|0-30, 낮을수록 인지 장애가 있음|FunctionalAssessment|기능평가|0-10, 낮을수록 기능 장애가 심함|
|MemoryComplaints|기억 장애 유무|0 : 없음, 1 : 예|BehavioralProblems|행동 문제 유무|0 : 없음, 1 : 예|
|ADL|일상생활활동 점수|0-10, 낮을수록 장애가 심함|

## 6. 증상
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|Confusion|혼란의 유무|0 : 없음, 1 : 예|Disorientation|방향 감각 상실 유무|0 : 없음, 1 : 예|
|PersonalityChanges|성격 변화 유무|0 : 없음, 1 : 예|DifficultyCompletingTasks|작업 완료 어려움 유무|0 : 없음, 1 : 예|
|Forgetfulness|건망증 유무|0 : 없음, 1 : 예|

## 7. 진단
|column 명|설명|value|column 명|설명|value|
|--|--|--|--|--|--|
|Diagnosis|알츠하이머 유무|0 : 없음, 1 : 예|DoctorInCharge|담당 의사에 대한 정보|'XXXConfid'|

In [1]:
pip install statsmodels


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imblearn


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pingouin


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from scipy import stats 
from statsmodels.formula.api import logit 
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency



df = pd.read_csv('../data/alzheimers_disease_data.csv')
df

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis,DoctorInCharge
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,0,0,1.725883,0,0,0,1,0,0,XXXConfid
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,0,0,2.592424,0,0,0,0,1,0,XXXConfid
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,0,0,7.119548,0,1,0,1,0,0,XXXConfid
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,0,1,6.481226,0,0,0,0,0,0,XXXConfid
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,0,0,0.014691,0,0,1,1,0,0,XXXConfid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,6895,61,0,0,1,39.121757,0,1.561126,4.049964,6.555306,...,0,0,4.492838,1,0,0,0,0,1,XXXConfid
2145,6896,75,0,0,2,17.857903,0,18.767261,1.360667,2.904662,...,0,1,9.204952,0,0,0,0,0,1,XXXConfid
2146,6897,77,0,0,1,15.476479,0,4.594670,9.886002,8.120025,...,0,0,5.036334,0,0,0,0,0,1,XXXConfid
2147,6898,78,1,3,1,15.299911,0,8.674505,6.354282,1.263427,...,0,0,3.785399,0,0,0,0,1,1,XXXConfid


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PatientID,2149.0,5825.000000,620.507185,4751.000000,5288.000000,5825.000000,6362.000000,6899.000000
Age,2149.0,74.908795,8.990221,60.000000,67.000000,75.000000,83.000000,90.000000
Gender,2149.0,0.506282,0.500077,0.000000,0.000000,1.000000,1.000000,1.000000
Ethnicity,2149.0,0.697534,0.996128,0.000000,0.000000,0.000000,1.000000,3.000000
EducationLevel,2149.0,1.286645,0.904527,0.000000,1.000000,1.000000,2.000000,3.000000
BMI,2149.0,27.655697,7.217438,15.008851,21.611408,27.823924,33.869778,39.992767
Smoking,2149.0,0.288506,0.453173,0.000000,0.000000,0.000000,1.000000,1.000000
AlcoholConsumption,2149.0,10.039442,5.757910,0.002003,5.139810,9.934412,15.157931,19.989293
PhysicalActivity,2149.0,4.920202,2.857191,0.003616,2.570626,4.766424,7.427899,9.987429
DietQuality,2149.0,4.993138,2.909055,0.009385,2.458455,5.076087,7.558625,9.998346


# 데이터 전처리

## 결측치 확인 : 없음

In [6]:
df.isnull().sum()

PatientID                    0
Age                          0
Gender                       0
Ethnicity                    0
EducationLevel               0
BMI                          0
Smoking                      0
AlcoholConsumption           0
PhysicalActivity             0
DietQuality                  0
SleepQuality                 0
FamilyHistoryAlzheimers      0
CardiovascularDisease        0
Diabetes                     0
Depression                   0
HeadInjury                   0
Hypertension                 0
SystolicBP                   0
DiastolicBP                  0
CholesterolTotal             0
CholesterolLDL               0
CholesterolHDL               0
CholesterolTriglycerides     0
MMSE                         0
FunctionalAssessment         0
MemoryComplaints             0
BehavioralProblems           0
ADL                          0
Confusion                    0
Disorientation               0
PersonalityChanges           0
DifficultyCompletingTasks    0
Forgetfu

## 필요없는 column 제거 : PatientID, DoctorInCharge

In [7]:
df.drop(['PatientID','DoctorInCharge'],axis=1, inplace=True)

In [8]:
cate_columns = ['Gender','Ethnicity','EducationLevel','Smoking','FamilyHistoryAlzheimers','CardiovascularDisease','Diabetes','Depression','HeadInjury','Hypertension','MemoryComplaints','BehavioralProblems','Confusion','Disorientation','PersonalityChanges','DifficultyCompletingTasks','Forgetfulness']
cate_indices = [df.columns.get_loc(col) for col in cate_columns]
cate_indices

[1, 2, 3, 5, 10, 11, 12, 13, 14, 15, 24, 25, 27, 28, 29, 30, 31]

In [9]:
from imblearn.over_sampling import SMOTENC
import pandas as pd
cate_columns = ['Gender','Ethnicity','EducationLevel','Smoking','FamilyHistoryAlzheimers','CardiovascularDisease','Diabetes','Depression','HeadInjury','Hypertension','MemoryComplaints','BehavioralProblems','Confusion','Disorientation','PersonalityChanges','DifficultyCompletingTasks','Forgetfulness']
cate_indices = [df.columns.get_loc(col) for col in cate_columns]
cate_indices

# SMOTENC 적용
overSampling = SMOTENC(categorical_features=cate_indices, sampling_strategy=0.8, random_state=42)
feature = df.drop('Diagnosis', axis=1)
target = df['Diagnosis']
feature_sample, target_sample = overSampling.fit_resample(feature, target)

print(feature.shape, target.shape, feature_sample.shape, target_sample.shape)
df = pd.concat([feature_sample, target_sample], axis=1)
df.


SyntaxError: invalid syntax (2333111181.py, line 15)

# 가설 검정 

## 1. 인적 사항, 병력, 일상 생활, 임상 측정, 인지 및 기능 평가, 증상이 알츠하이머 유/무와 상관관계가 있다. 
- ### 결과
    - MemoryComplaints가 없을 때에 비해 MemoryComplaints가 있을 때 알츠하이머가 있을 가능성이 약 36.1% 상승함(27.9% -> 64%)
    - BehavioralProblems가 없을 때에 비해 BehavioralProblems가 있을 때 알츠하이머가 있을 가능성이 약 29,4% 상승함(30.8% -> 60.2%)
    - 알츠하이머가 있는 그룹이 없는 그룹에 비해 MMSE 점수가 5.58점 낮음.
    - 알츠하이머가 있는 그룹이 없는 그룹에 비해 FunctionalAssessment 점수가 2.95점 낮음.
    - 알츠하이머가 있는 그룹이 없는 그룹에 비해 ADL 점수가 2.90점 낮음.
    - 알츠하이머가 있는 그룹이 없는 그룹에 비해 SleepQuality 점수가 0.33점 낮음.

- ### 결론
    - 인지 및 기능 평가 그룹(MemoryComplaints, BehavioralProblems, MMSE, FunctionalAssessment, ADL)이 알츠하이머 유/무와 상관관계가 있음.
    - 인지 및 기능 평가 그룹 외에도 SleepQuality도 알츠하이머 유/무와 상관관계가 있지만, 큰 차이는 없음.

### 범주형 변수와 알츠하이머 유/무의 상관관계 분석
#### - 카이제곱 검정을 통해 각 변수가 알츠하이머 유/무와 상관관계가 있는지 분석 
#### 범주형 변수
- 'Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension', 'MemoryComplaints', 'BehavioralProblems', 'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness'

In [10]:
cate_columns = ['Gender','Ethnicity','EducationLevel','Smoking','FamilyHistoryAlzheimers','CardiovascularDisease','Diabetes','Depression','HeadInjury','Hypertension','MemoryComplaints','BehavioralProblems','Confusion','Disorientation','PersonalityChanges','DifficultyCompletingTasks','Forgetfulness']


In [11]:
for i in cate_columns:
    cross_table = pd.crosstab(df[i],df['Diagnosis'])
    chi2,pvalue,dof,expected = chi2_contingency(cross_table)
    if pvalue <= 0.05:
        print(i + '는 알츠하이머 유/무와 연관성이 있습니다.')
        print('검정통계량 :',chi2)
        print(cross_table)
        print('')
    

MemoryComplaints는 알츠하이머 유/무와 연관성이 있습니다.
검정통계량 : 200.62370423042574
Diagnosis            0    1
MemoryComplaints           
0                 1228  474
1                  161  286

BehavioralProblems는 알츠하이머 유/무와 연관성이 있습니다.
검정통계량 : 106.8792165385845
Diagnosis              0    1
BehavioralProblems           
0                   1255  557
1                    134  203



### 수치형 변수와 알츠하이머 유/무의 상관관계 분석
#### - 독립표본의 차이 검정을 통해 각 변수가 알츠하이머 유/무와 상관관계가 있는지 분석
#### 수치형 변수
- 'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides', 'MMSE', 'FunctionalAssessment', 'ADL'

In [12]:
num_columns = []
for i in list(df.columns):
    if i not in cate_columns and i != 'Diagnosis':
        num_columns.append(i)

In [13]:
for i in num_columns:
    sample_1 = df.loc[df['Diagnosis']==0,i]
    sample_2 = df.loc[df['Diagnosis']==1,i]
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:

        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False

        ## t 검정
        statistic, pvalue = stats.ttest_ind(sample_1,sample_2,equal_var = equal_var)
        if pvalue <= 0.05:
            print(i,"는 알츠하이머 유/무와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)
    else:

        ## Mann-Whitney U 검정
        statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2)
        if pvalue <= 0.05:
            print(i,"는 알츠하이머 유/무와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)

SleepQuality 는 알츠하이머 유/무와 연관성이 있습니다.
검정 통계량 : 563558.0
CholesterolHDL 는 알츠하이머 유/무와 연관성이 있습니다.
검정 통계량 : 500704.0
MMSE 는 알츠하이머 유/무와 연관성이 있습니다.
검정 통계량 : 678416.0
FunctionalAssessment 는 알츠하이머 유/무와 연관성이 있습니다.
검정 통계량 : 761542.0
ADL 는 알츠하이머 유/무와 연관성이 있습니다.
검정 통계량 : 738445.0


In [14]:
for i in ['SleepQuality','CholesterolHDL','MMSE','FunctionalAssessment','ADL']:
    sample_1 = df.loc[df['Diagnosis']==0,i]
    sample_2 = df.loc[df['Diagnosis']==1,i]
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:

        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False

        ## t 검정
        statistic, pvalue = stats.ttest_ind(sample_1,sample_2,alternative= 'greater', equal_var = equal_var)
        if pvalue <= 0.05:
            print(i,"점수가 낮을수록, 알츠하이머가 있을 확률이 올라갑니다.")
            print(sample_1.mean() - sample_2.mean())
            print('검정 통계량 :',statistic)
            print('')
    else:

        ## Mann-Whitney U 검정
        statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2,alternative= 'greater')
        if pvalue <= 0.05:
            print(i,"점수가 낮을수록, 알츠하이머가 있을 확률이 올라갑니다.")
            print(sample_1.median() - sample_2.median())
            print('검정 통계량 :',statistic)
            print('')

SleepQuality 점수가 낮을수록, 알츠하이머가 있을 확률이 올라갑니다.
0.3263372104112845
검정 통계량 : 563558.0

MMSE 점수가 낮을수록, 알츠하이머가 있을 확률이 올라갑니다.
5.579962493708555
검정 통계량 : 678416.0

FunctionalAssessment 점수가 낮을수록, 알츠하이머가 있을 확률이 올라갑니다.
2.945239901790942
검정 통계량 : 761542.0

ADL 점수가 낮을수록, 알츠하이머가 있을 확률이 올라갑니다.
2.899369869252565
검정 통계량 : 738445.0



## 2. 인적 사항, 증상, 병력, 일상 생활, 임상 측정이 인지 및 기능 평가의 변수들과 상관관계가 있다. 
- ### 결과
    - DifficultyCompletingTasks가 MemoryComplaints와 관련이 있다.
        - DifficultyCompletingTasks가 없을 때에 비해 DifficultyCompletingTasks가 있을 때 MemoryComplaints가 있을 가능성이 약 4.9% 상승함(20.03% -> 24.93%)
        - MemoryComplaints가 없을 때에 비해 MemoryComplaints가 있을 때 DifficultyCompletingTasks가 있을 가능성이 약 4.0% 상승함(15.04% -> 19.02%)
    - HeadInjury가 BehavioralProblems와 관련이 있다.
        - HeadInjury가 없을 때에 비해 HeadInjury가 있을 때 BehavioralProblems가 있을 가능성이 약 5.94% 상승함(15.15% -> 21.09%)
        - BehavioralProblems가 없을 때에 비해 BehavioralProblems가 있을 때 HeadInjury가 있을 가능성이 약 3.8% 상승함(8.66% -> 12.46%)

- ### 결론
    - 기능 평가 결과 기억력에 문제가 있을 경우 업무 완료를 어려워하는 증상이 나타날 수 있다.
    - 두부 손상을 당한 병력이 있으면 기능 평가에서 행동에 문제가 있을 수 있다.

### 데이터 준비

In [15]:
bi_cate_columns = ['Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',  'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness']
poly_cate_columns = ['Ethnicity','EducationLevel']

### FunctionalAssessment

#### 범주형 변수와 FunctionalAssessment의 상관관계 분석
##### - 범주형 변수가 그룹이 2개일 경우 T-test/비모수 검정, 세 개 이상일 경우 일원분산분석/크루스칼-왈리스 검정을 통해 각 변수가 FunctionalAssessment와 상관관계가 있는지 분석 
##### 이항 변수
- 'Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',  'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness'
##### 다항 변수
- 'Ethnicity','EducationLevel'

In [36]:
import pingouin
for i in bi_cate_columns:
    sample_1 = df.loc[df[i]==0,'FunctionalAssessment']
    sample_2 = df.loc[df[i]==1,'FunctionalAssessment']
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:

        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False

        ## t 검정
        statistic, pvalue = stats.ttest_ind(sample_1,sample_2,equal_var = equal_var)
        if pvalue <= 0.05:
            print(i,"는 'FunctionalAssessment'와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)
    else:

        ## Mann-Whitney U 검정
        statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2)
        if pvalue <= 0.05:
            print(i,"는 'FunctionalAssessment'와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)
for i in poly_cate_columns:
    sample_1 = df.loc[df[i]==0,'FunctionalAssessment']
    sample_2 = df.loc[df[i]==1,'FunctionalAssessment']
    sample_3 = df.loc[df[i]==2,'FunctionalAssessment']
    sample_4 = df.loc[df[i]==3,'FunctionalAssessment']
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    statistic3,pvalue3 = stats.shapiro(sample_3)
    statistic4,pvalue4 = stats.shapiro(sample_4)
    if pvalue1 > 0.05 and pvalue2 > 0.05 and pvalue3 > 0.05 and pvalue4 > 0.05:
    
        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2,sample_3,sample_4)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False
    
        ## t 검정
        if equal_var == True:
            statistic, pvalue = stats.f_oneway(sample_1,sample_2,sample_3,sample_4)
            if pvalue <= 0.05:
                print(0,':',sample_1.mean())
                print(1,':',sample_2.mean())
                print(2,':',sample_3.mean())
                print(3,':',sample_4.mean())
                print(i,"는 'FunctionalAssessment'와 연관성이 있습니다.")
                print('검정 통계량 :',statistic)
        else:
            statistic, pvalue = pingouin.welch_anova(dv='value',between='target',data=pd.concat([sample_1, sample_2, sample_3, sample_4]))
            if pvalue <= 0.05:
                print(0,':',sample_1.mean())
                print(1,':',sample_2.mean())
                print(2,':',sample_3.mean())
                print(3,':',sample_4.mean())
                print(i,"는 'FunctionalAssessment'와 연관성이 있습니다.")
                print('검정 통계량 :',statistic)
            
    else:
    
        ## Mann-Whitney U 검정
        statistic, pvalue = stats.kruskal(sample_1,sample_2,sample_3,sample_4)
        if pvalue <= 0.05:
            print(i,"는 'FunctionalAssessment'와 연관성이 있습니다.")
            print(0,':',sample_1.median())
            print(1,':',sample_2.median())
            print(2,':',sample_3.median())
            print(3,':',sample_4.median())
            print('검정 통계량 :',statistic)

Diabetes 는 'FunctionalAssessment'와 연관성이 있습니다.
검정 통계량 : 274656.0


In [37]:
sample_1 = df.loc[df['Smoking']==0,'FunctionalAssessment']
sample_2 = df.loc[df['Smoking']==1,'FunctionalAssessment']

## 정규성 검정
statistic1,pvalue1 = stats.shapiro(sample_1)
statistic2,pvalue2 = stats.shapiro(sample_2)
if pvalue1 > 0.05 and pvalue2 > 0.05:

    ## 등분산 검정
    statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
    if pvalues_levene > 0.05:
        equal_var = True
    else:
        equal_var = False

    ## t 검정
    statistic, pvalue = stats.ttest_ind(sample_1,sample_2,alternative='greater', equal_var = equal_var)
    if pvalue <= 0.05:
        print("Smoking가 있을 경우 'FunctionalAssessment'점수가 더 낮아집니다.")
        print(sample_1.mean() - sample_2.mean())
        print('검정 통계량 :',statistic)
else:

    ## Mann-Whitney U 검정
    statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2,alternative='greater')
    if pvalue <= 0.05:
        print("Smoking가 있을 경우 'FunctionalAssessment'점수가 더 낮아집니다.")
        print(sample_1.median() - sample_2.median())
        print('검정 통계량 :',statistic)

#### 수치형 변수와 FunctionalAssessment의 상관관계 분석
##### - 상관관계 분석을 통해 각 변수가 FunctionalAssessment와 상관관계가 있는지 분석 
#####  수치형 변수
- 'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides'

In [38]:
FA_num_columns = ['Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides']

In [39]:
sample1 = df['FunctionalAssessment']
statistic1,pvalue1 = stats.shapiro(sample1)

for i in FA_num_columns:
    sample2 = df[i]
    statistic2,pvalue2 = stats.shapiro(sample2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:
        statistic,pvalue = stats.pearsonr(sample1,sample2)
        if pvalue <= 0.05:
            print(i,'는 FunctionalAssessment와 연관성이 있습니다.')
            print('상관계수 :',statistic)
            print('')
    else:
        statistic,pvalue = stats.spearmanr(sample1,sample2)
        if pvalue <= 0.05:
            print(i,'는 FunctionalAssessment와 연관성이 있습니다.')
            print('상관계수 :',statistic)
            print('')
            
    
    

### ADL

#### 범주형 변수와 ADL 상관관계 분석
##### - 범주형 변수가 그룹이 2개일 경우 T-test/비모수 검정, 세 개 이상일 경우 일원분산분석/크루스칼-왈리스 검정을 통해 각 변수가 ADL와 상관관계가 있는지 분석 
##### 이항 변수
- 'Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',  'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness'
##### 다항 변수
- 'Ethnicity','EducationLevel'

In [40]:
bi_cate_columns = ['Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',  'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness']
poly_cate_columns = ['Ethnicity','EducationLevel']

In [41]:
import pingouin
for i in bi_cate_columns:
    sample_1 = df.loc[df[i]==0,'ADL']
    sample_2 = df.loc[df[i]==1,'ADL']
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:

        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False

        ## t 검정
        statistic, pvalue = stats.ttest_ind(sample_1,sample_2,equal_var = equal_var)
        if pvalue <= 0.05:
            print(i,"는 'ADL'와 연관성이 있습니다.")
            print(sample_1.mean() - sample_2.mean())
            print('mean')
            print('검정 통계량 :',statistic)
    else:

        ## Mann-Whitney U 검정
        statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2)
        if pvalue <= 0.05:
            print(i,"는 'ADL'와 연관성이 있습니다.")
            print(sample_1.median() - sample_2.median())
            print('검정 통계량 :',statistic)
for i in poly_cate_columns:
    sample_1 = df.loc[df[i]==0,'ADL']
    sample_2 = df.loc[df[i]==1,'ADL']
    sample_3 = df.loc[df[i]==2,'ADL']
    sample_4 = df.loc[df[i]==3,'ADL']
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    statistic3,pvalue3 = stats.shapiro(sample_3)
    statistic4,pvalue4 = stats.shapiro(sample_4)
    if pvalue1 > 0.05 and pvalue2 > 0.05 and pvalue3 > 0.05 and pvalue4 > 0.05:
    
        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2,sample_3,sample_4)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False
    
        ## t 검정
        if equal_var == True:
            statistic, pvalue = stats.f_oneway(sample_1,sample_2,sample_3,sample_4)
            if pvalue <= 0.05:
                print(i,"는 'ADL'와 연관성이 있습니다.")
                print(0,':',sample_1.mean())
                print(1,':',sample_2.mean())
                print(2,':',sample_3.mean())
                print(3,':',sample_4.mean())
                print('검정 통계량 :',statistic)
        else:
            statistic, pvalue = pingouin.welch_anova(dv='value',between='target',data=pd.concat([sample_1, sample_2, sample_3, sample_4]))
            if pvalue <= 0.05:
                print(i,"는 'ADL'와 연관성이 있습니다.")
                print(0,':',sample_1.mean())
                print(1,':',sample_2.mean())
                print(2,':',sample_3.mean())
                print(3,':',sample_4.mean())
                print('검정 통계량 :',statistic)
            
    else:
    
        ## Mann-Whitney U 검정
        statistic, pvalue = stats.kruskal(sample_1,sample_2,sample_3,sample_4)
        if pvalue <= 0.05:
            print(i,"는 'ADL'와 연관성이 있습니다.")
            print(0,':',sample_1.median())
            print(1,':',sample_2.median())
            print(2,':',sample_3.median())
            print(3,':',sample_4.median())
            print('검정 통계량 :',statistic)

#### 수치형 변수와 ADL의 상관관계 분석
##### - 상관관계 분석을 통해 각 변수가 ADL와 상관관계가 있는지 분석 
#####  수치형 변수
- 'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides'

In [42]:
ADL_num_columns = ['Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides']

In [43]:
sample1 = df['ADL']
statistic1,pvalue1 = stats.shapiro(sample1)

for i in ADL_num_columns:
    sample2 = df[i]
    statstic2,pvalue2 = stats.shapiro(sample2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:
        statistic,pvalue = stats.pearsonr(sample1,sample2)
        if pvalue <= 0.05:
            print(i,'는 ADL와 연관성이 있습니다.')
            print('상관계수 :',statistic)
            print('')
    else:
        statistic,pvalue = stats.spearmanr(sample1,sample2)
        if pvalue <= 0.05:
            print(i,'는 ADL와 연관성이 있습니다.')
            print('상관계수 :',statistic)
            print('')
            
    
    

### MMSE

#### 범주형 변수와 MMSE 상관관계 분석
##### - 범주형 변수가 그룹이 2개일 경우 T-test/비모수 검정, 세 개 이상일 경우 일원분산분석/크루스칼-왈리스 검정을 통해 각 변수가 MMSE와 상관관계가 있는지 분석 
##### 이항 변수
- 'Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',  'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness'
##### 다항 변수
- 'Ethnicity','EducationLevel'

In [44]:
bi_cate_columns = ['Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',  'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness']
poly_cate_columns = ['Ethnicity','EducationLevel']

In [45]:
import pingouin
for i in bi_cate_columns:
    sample_1 = df.loc[df[i]==0,'MMSE']
    sample_2 = df.loc[df[i]==1,'MMSE']
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:

        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False

        ## t 검정
        statistic, pvalue = stats.ttest_ind(sample_1,sample_2,equal_var = equal_var)
        if pvalue <= 0.05:
            print(i,"는 'MMSE'와 연관성이 있습니다.")
            print(sample_1.mean() - sample_2.mean())
            print('검정 통계량 :',statistic)
    else:

        ## Mann-Whitney U 검정
        statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2)
        if pvalue <= 0.05:
            print(i,"는 'MMSE'와 연관성이 있습니다.")
            print(sample_1.median() - sample_2.median())
            print('검정 통계량 :',statistic)
for i in poly_cate_columns:
    sample_1 = df.loc[df[i]==0,'MMSE']
    sample_2 = df.loc[df[i]==1,'MMSE']
    sample_3 = df.loc[df[i]==2,'MMSE']
    sample_4 = df.loc[df[i]==3,'MMSE']
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    statistic3,pvalue3 = stats.shapiro(sample_3)
    statistic4,pvalue4 = stats.shapiro(sample_4)
    if pvalue1 > 0.05 and pvalue2 > 0.05 and pvalue3 > 0.05 and pvalue4 > 0.05:
    
        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2,sample_3,sample_4)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False
    
        ## t 검정
        if equal_var == True:
            statistic, pvalue = stats.f_oneway(sample_1,sample_2,sample_3,sample_4)
            if pvalue <= 0.05:
                print(i,"는 'MMSE'와 연관성이 있습니다.")
                print(0,':',sample_1.mean())
                print(1,':',sample_2.mean())
                print(2,':',sample_3.mean())
                print(3,':',sample_4.mean())
                print('검정 통계량 :',statistic)
        else:
            statistic, pvalue = pingouin.welch_anova(dv='value',between='target',data=pd.concat([sample_1, sample_2, sample_3, sample_4]))
            if pvalue <= 0.05:
                print(i,"는 'MMSE'와 연관성이 있습니다.")
                print(0,':',sample_1.mean())
                print(1,':',sample_2.mean())
                print(2,':',sample_3.mean())
                print(3,':',sample_4.mean())
                
                print('검정 통계량 :',statistic)
            
    else:
    
        ## Mann-Whitney U 검정
        statistic, pvalue = stats.kruskal(sample_1,sample_2,sample_3,sample_4)
        if pvalue <= 0.05:
            print(i,"는 'MMSE'와 연관성이 있습니다.")
            print(0,':',sample_1.median())
            print(1,':',sample_2.median())
            print(2,':',sample_3.median())
            print(3,':',sample_4.median())
            print('검정 통계량 :',statistic)

#### 수치형 변수와 MMSE 상관관계 분석
##### - 상관관계 분석을 통해 각 변수가 MMSE와 상관관계가 있는지 분석 
#####  수치형 변수
- 'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides'

In [46]:
MMSE_num_columns = ['Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides']

In [47]:
sample1 = df['MMSE']
statistic1,pvalue1 = stats.shapiro(sample1)

for i in MMSE_num_columns:
    sample2 = df[i]
    statstic2,pvalue2 = stats.shapiro(sample2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:
        statistic,pvalue = stats.pearsonr(sample1,sample2)
        if pvalue <= 0.05:
            print(i,'는 MMSE와 연관성이 있습니다.')
            print('상관계수 :',statistic)
            print('')
            print('pearsonr')
    else:
        statistic,pvalue = stats.spearmanr(sample1,sample2)
        if pvalue <= 0.05:
            print(i,'는 MMSE와 연관성이 있습니다.')
            print('상관계수 :',statistic)
            print('')
            
    
    

### MemoryComplaints
- DifficultyCompletingTasks가 없을 때에 비해 DifficultyCompletingTasks가 있을 때 MemoryComplaints가 있을 가능성이 약 4.9% 상승함(20.03% -> 24.93%)
- MemoryComplaints가 없을 때에 비해 MemoryComplaints가 있을 때 DifficultyCompletingTasks가 있을 가능성이 약 4.0% 상승함(15.04% -> 19.02%)


#### 범주형 변수와 MemoryComplaints의 상관관계 분석
##### - 카이제곱 검정을 통해 각 변수가 MemoryComplaints와 상관관계가 있는지 분석 
##### 범주형 변수
- 'Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension', 'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness'

In [48]:
MC_cate_columns = ['Gender','Ethnicity','EducationLevel','Smoking','FamilyHistoryAlzheimers','CardiovascularDisease','Diabetes','Depression','HeadInjury','Hypertension','Confusion','Disorientation','PersonalityChanges','DifficultyCompletingTasks','Forgetfulness']


In [49]:
for i in MC_cate_columns:
    cross_table = pd.crosstab(df[i],df['MemoryComplaints'])
    chi2,pvalue,dof,expected = chi2_contingency(cross_table)
    if pvalue <= 0.05:
        print(i + '는 MemoryComplaints와 연관성이 있습니다.')
        print('검정통계량 :',chi2)
        print(cross_table)

DifficultyCompletingTasks는 MemoryComplaints와 연관성이 있습니다.
검정통계량 : 3.896689659224866
MemoryComplaints              0    1
DifficultyCompletingTasks           
0                          1446  362
1                           256   85


#### 수치형 변수와 MemoryComplaints의 상관관계 분석
##### - 독립표본의 차이 검정을 통해 각 변수가 MemoryComplaints와 상관관계가 있는지 분석
##### 수치형 변수
- 'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides'

In [50]:
MC_num_columns = ['Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides']

In [51]:
for i in MC_num_columns:
    sample_1 = df.loc[df['MemoryComplaints']==0,i]
    sample_2 = df.loc[df['MemoryComplaints']==1,i]
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:

        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False

        ## t 검정
        statistic, pvalue = stats.ttest_ind(sample_1,sample_2,equal_var = equal_var)
        if pvalue <= 0.05:
            print(i,"는 MemoryComplaints와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)
    else:

        ## Mann-Whitney U 검정
        statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2)
        if pvalue <= 0.05:
            print(i,"는 MemoryComplaints와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)

### BehavioralProblems
- HeadInjury가 없을 때에 비해 HeadInjury가 있을 때 BehavioralProblems가 있을 가능성이 약 5.94% 상승함(15.15% -> 21.09%)
- BehavioralProblems가 없을 때에 비해 BehavioralProblems가 있을 때 HeadInjury가 있을 가능성이 약 3.8% 상승함(8.66% -> 12.46%)


#### 범주형 변수와 BehavioralProblems의 상관관계 분석
##### - 카이제곱 검정을 통해 각 변수가 BehavioralProblems와 상관관계가 있는지 분석 
##### 범주형 변수
- 'Gender', 'Ethnicity', 'EducationLevel', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease', 'Diabetes', 'Depression', 'HeadInjury', 'Hypertension', 'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness'

In [52]:
BP_cate_columns = ['Gender','Ethnicity','EducationLevel','Smoking','FamilyHistoryAlzheimers','CardiovascularDisease','Diabetes','Depression','HeadInjury','Hypertension','Confusion','Disorientation','PersonalityChanges','DifficultyCompletingTasks','Forgetfulness']


In [53]:
for i in BP_cate_columns:
    cross_table = pd.crosstab(df[i],df['BehavioralProblems'])
    chi2,pvalue,dof,expected = chi2_contingency(cross_table)
    if pvalue <= 0.05:
        print(i + '는 BehavioralProblems와 연관성이 있습니다.')
        print('검정통계량 :',chi2)
        print(cross_table)

HeadInjury는 BehavioralProblems와 연관성이 있습니다.
검정통계량 : 4.437624106725124
BehavioralProblems     0    1
HeadInjury                   
0                   1655  295
1                    157   42


#### 수치형 변수와 BehavioralProblems의 상관관계 분석
##### - 독립표본의 차이 검정을 통해 각 변수가 BehavioralProblems와 상관관계가 있는지 분석
##### 수치형 변수
- 'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides'

In [54]:
BP_num_columns = ['Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides']

In [55]:
for i in BP_num_columns:
    sample_1 = df.loc[df['BehavioralProblems']==0,i]
    sample_2 = df.loc[df['BehavioralProblems']==1,i]
    
    ## 정규성 검정
    statistic1,pvalue1 = stats.shapiro(sample_1)
    statistic2,pvalue2 = stats.shapiro(sample_2)
    if pvalue1 > 0.05 and pvalue2 > 0.05:

        ## 등분산 검정
        statistic_levene,pvalues_levene = stats.levene(sample_1,sample_2)
        if pvalues_levene > 0.05:
            equal_var = True
        else:
            equal_var = False

        ## t 검정
        statistic, pvalue = stats.ttest_ind(sample_1,sample_2,equal_var = equal_var)
        if pvalue <= 0.05:
            print(i,"는 BehavioralProblems와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)
    else:

        ## Mann-Whitney U 검정
        statistic, pvalue = stats.mannwhitneyu(sample_1,sample_2)
        if pvalue <= 0.05:
            print(i,"는 BehavioralProblems와 연관성이 있습니다.")
            print('검정 통계량 :',statistic)

# 결론 
## 1. 인지 및 기능 평가 그룹이 알츠하이머 유/무와 상관관계가 있기 때문에 이 그룹을 통해 알츠하이머 유/무를 예측하는 모델을 제작해야겠다. 
## 2. 두부 손상과 업무 완료 어려움이 각각 기능 평가와 기억력에 영향을 줄 수 있다. 60세 이상의 환자가 두부 손상을 당한 병력이 있거나 평소 업무 완료 어려움을 겪는다면 인지 및 기능 평가를 통해 알츠하이머 검사를 진행하는 것을 추천해줄 수 있다.